---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [37]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [47]:
def answer_one():
    # Langkah#1 - Find max values of gold medal
    max_gold_summer = df['Gold'].max()  
    
    # Langkah#2 - Temukan negara dengan jumlah gold = max

    # Alternative#1: Karena hasil yang diminta string, kita ambil hasil dari 'ID'
    #country_id = df['ID'][df['Gold'] == max_gold_summer].item()
    #return country_id
    
    # Alternatif#2: Refactor cara yang efisien. Dapatkan nama negara dari index baris
    #country = df.where(df['Gold'] == max_gold_summer)
    #country = country.dropna()  # Hapus hasil NaN dari table
    #country_name = country.index.values[0]  # Index value dari baris ini adalah "United States". Untuk mendapatkannya, kita pakai `.index.values[0]`. Sumber: https://thispointer.com/python-pandas-how-to-get-column-and-row-names-in-dataframe/
    #return country_name
    
    # Alternative#3: This will produce USA with all columns
    country = df.loc[df['Gold'] == max_gold_summer]
    country_name = country.index.values[0]  # Index value dari baris ini adalah "United States". Untuk mendapatkannya, kita pakai `.index.values[0]`.
    
    return country_name

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [55]:
def answer_two():
    # Alternative#1 - Duplikat tabel untuk jaga-jaga siapa tau tabelnya jadi rusak
    # Langkah#1 - Dapatkan selisih gold dari kedua musim
    #summer_gold = df['Gold']
    #winter_gold = df['Gold.1']
    #diff = summer_gold - winter_gold  
    
    # Langkah#2 - Tambahkan hasil diff pada tabel
    #copy_df = df.copy() 
    #copy_df['Diff'] = diff.abs() # Tambah kolom 'Diff' pada tabel dengan nilai selish yang sudah di positifkan    
    
    # Langkah#3 - Dapatkan nama negara dengan nilai diff(selisih gold dari kedua musim) terbesar
    #max_diff = diff.abs().max()
    #country = copy_df['ID'][copy_df['Diff'] == max_diff].item()  # Karena hasil yang diminta string, kita ambil hasil dari 'ID'
    
    #return country
    
    # ---------------------

    # Alternative#2: Langsung tambah kolom `Diff` ada tabel existing
    # Langkah#1 - Dapatkan selisih gold dari kedua musim
    summer_gold = df['Gold']
    winter_gold = df['Gold.1']
    diff = summer_gold - winter_gold  
    
    # Langkah#2 - Tambahkan hasil diff pada tabel
    df['Diff'] = diff.abs()
    
    # Langkah#3 - Dapatkan nama negara dengan nilai diff(selisih gold dari kedua musim) terbesar
    max_diff = diff.abs().max()
    country = df.loc[df['Diff'] == max_diff]  # Dapatkan nama negara dari index baris
    country_name = country.index.values[0]  # Index value dari baris ini adalah "United States". Untuk mendapatkannya, kita pakai `.index.values[0]`.
    
    return country_name

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [2]:
def answer_three():
    # Langkah#1 - Dapatkan negara yang mendapatkan paling tidak 1 gold dalam summer dan winter
    
    # Alternatif#1: Cari negara yang mendapatkan emas pada masing2 musim --> gak jadi pake karena bingung cara gabunginnya
    # one_gold_summer = df['Gold'][df['Gold'] > 0]
    # one_gold_winter = df['Gold.1'][df['Gold.1'] > 0]
    # diff = abs(one_gold_summer - one_gold_winter)
    
    # Alternatif#2: Pilih negara dengan langsung menyaring pada kedua musim
    one_gold = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    
    # check row --> one_gold.shape[0]
    # check column --> one_gold.shape[1]
    
    # Langkah#2 - Hitung perbandingan
    summer_gold = one_gold['Gold']
    winter_gold = one_gold['Gold.1']
    total_gold = one_gold['Gold.2']
    diff = (summer_gold - winter_gold) / total_gold  # Hitung selisih emas berdasarkan rumus diatas
    
    # Langkah#3 - Dapatkan nama negara dengan nilai diff(selisih gold dari kedua musim) terbesar
    
    # Alternatif#1: Karena hasil yang diminta string, kita ambil hasil dari 'ID'. Serta kita lakukan duplikasi tabel
    #one_gold_countries = one_gold['ID'].copy() # Just copy 'ID' column --> failed when we add new 'Diff' column
    #one_gold_countries = one_gold[['ID']].copy() # Just copy 'ID' column --> Ref: https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy
    
    # Alternatif#2: Langsung tambah kolom `Diff` ada tabel existing
    df['Diff'] = diff.abs()  # Tambahkan hasil diff pada tabel
    max_diff = diff.abs().max()  # Dapatkan nilai max dari semua diff
    country = df.loc[df['Diff'] == max_diff]  # Dapatkan nama negara dari index baris
    #country = country.dropna()  # Hapus hasil NaN dari table
    country_name = country.index.values[0]  # Index value dari baris ini adalah "Bulgaria". Untuk mendapatkannya, kita pakai `.index.values[0]`. 
    
    return country_name

answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [73]:
def answer_four():
    gold_point = df['Gold.2']*3
    silver_point = df['Silver.2']*2
    bronze_point = df['Bronze.2']*1
    points = gold_point + silver_point + bronze_point
    return points

answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [3]:
import pandas as pd
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [110]:
def answer_five():
    # Langkah#1 - Hitung jumlah County untuk setiap State

    # Berdasarkan dokumentasi pada link pdf, 
    # SUMLEV = 40 artinya baris tersebut adalah State
    # SUMLEV = 50 artinya baris tersebut adalah County
    
    """
    Perbedaan Series dan DataFrame
    Series --> berbentuk array
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html
    
    Contoh Series = pd.Series([4, 2, 0, 8], name='legs', index=idx)
    
    DataFrame --> berbentuk tabel
    https://www.geeksforgeeks.org/python-pandas-dataframe-max/
    
    Contoh DataFrame:
    df = pd.DataFrame({"A":[12, 4, 5, 44, 1], 
                   "B":[5, 2, 54, 3, 2], 
                   "C":[20, 16, 7, 3, 8],  
                   "D":[14, 3, 17, 2, 6]}) 
    """
    
    county = census_df[census_df['SUMLEV'] == 50]  # Ambil data County saja
    sum_county = county.groupby('STNAME').size()  # Hasilnya dalam bentuk Series --> https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
    
    # Langkah#2 - Temukan State dengan jumlah County terbanyak
    max_sum_county = sum_county.idxmax()  # Tampilkan index dari hasil max --> https://stackoverflow.com/questions/46577525/how-to-get-max-value-and-name-from-a-pandas-series
    
    return max_sum_county

answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [192]:
def answer_six():
    # Alternative#1: Pakai fungsi sorting
    # Langkah#1 - Filter data State saja
    #state = census_df[census_df['SUMLEV'] == 40]
    
    # Langkah#2 - Urutkan hasil dengan angka sensus terbesar
    """
    Referensi sorting:
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
    https://www.geeksforgeeks.org/python-pandas-dataframe-sort_values-set-1/
    https://datatofish.com/sort-pandas-dataframe/
    """
    #sorted_state = state.sort_values(by='CENSUS2010POP', ascending=False)
    
    # Langkah#3 - Tampilkan nama 3 State teratas
    #return sorted_state['STNAME'].head(3)

    # Alternative#2: Pakai fungsi nlargest
    # Langkah#1 - Filter data State saja
    state = census_df[census_df['SUMLEV'] == 40]
    
    # Langkah#2 - Dapatkan angka sensus terbesar
    state = state.nlargest(3, 'CENSUS2010POP')['STNAME']
    # Alternatif dapatkan state --> state.nlargest(3, 'CENSUS2010POP').loc[:, 'STNAME']
    list_state = state.values.tolist()  # Konversi hasilnya dalam bentuk list --> https://datatofish.com/convert-pandas-dataframe-to-list/
    
    return list_state

answer_six()

['California', 'Texas', 'New York']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [ ]:
def answer_seven():
    return "YOUR ANSWER HERE"

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [ ]:
def answer_eight():
    return "YOUR ANSWER HERE"